In [1]:
from datasets import load_from_disk
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, AdamW, DataCollatorForTokenClassification
import torch
from tqdm import tqdm
import torch.nn as nn

## look it was good step that i have saveed tokenized dataset , here i dont have to retokenized thema again
tokenized_datasets = load_from_disk("../data/tokenized_dataset")
train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]

# lets import pretrained model and tokeniezr 
model_checkpoint = "bert-base-cased" # please remember it is (-)between letters not ___ once i was stuck 3 hours on this error 
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=11)

## lets write data colletrla 
data_collator = DataCollatorForTokenClassification(tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(val_dataset, batch_size=8, collate_fn=data_collator)

#  i think ,my devise does not ahve any gpu and its ok for this project ebacsue data is not as large 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#  lets import optimizer 
optimizer = AdamW(model.parameters(), lr=5e-5)

# I am going to write my own training loop instead of using training argument beacsue it provides more control, when I ahve to handle big data like in real life i ahve to write my own loop 
def train_one_epoch(model, dataloader, optimizer):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            total_loss += outputs.loss.item()
    return total_loss / len(dataloader)

# this is final step 
epochs = 3
for epoch in range(epochs):
    train_loss = train_one_epoch(model, train_dataloader, optimizer)
    val_loss = evaluate(model, val_dataloader)
    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")



# Save model and tokenizer to outputs folder
output_dir = "../outputs/ner_model"

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f" Model and tokenizer saved to '{output_dir}'")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\hamza\miniconda3\Lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 63/63 [02:04<00:00,  1.98s/it]


Epoch 1 | Train Loss: 0.2647 | Val Loss: 1.7008


100%|██████████| 63/63 [02:50<00:00,  2.71s/it]


Epoch 2 | Train Loss: 0.0045 | Val Loss: 1.7434


100%|██████████| 63/63 [03:35<00:00,  3.42s/it]


Epoch 3 | Train Loss: 0.0016 | Val Loss: 1.8014
 Model and tokenizer saved to '../outputs/ner_model'
